In [52]:
import numpy as np, pandas as pd, os, pickle
from natsort import natsorted
from scipy.signal import find_peaks
from datetime import timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output


print('Header files imported......')

Header files imported......


In [53]:
def plot_sensor_active_periods(data_subject_2, environmental_sensors, st, et):
    if st != None and et!=None:
        print('I came inside......')
        start_date = st #pd.to_datetime('2023-07-21 00:00:00')
        end_date = et #pd.to_datetime('2023-07-26 00:00:00')

    df_combined_non_temp_humidity = []
    df_combined_temp_humidity = {}
    for sens in range(len(environmental_sensors)):
        sensor_name = environmental_sensors[sens]
        
        print(sens, sensor_name)
        if sensor_name in ['Stove_Hum_Temp_humidity','Stove_Hum_Temp_temp']:
            df_sensor = data_subject_2[sensor_name][0]
            df_sensor['ts_datetime'] = pd.to_datetime(df_sensor['ts_datetime'])

            if st != None and et != None:
                filtered_df = df_sensor[(df_sensor['ts_datetime'] >= start_date) & (df_sensor['ts_datetime'] <= end_date)]
            else:
                filtered_df = df_sensor.copy()

            sub_df_sensor = filtered_df[['sensor_id', 'ts_datetime', 'sensor_status']]
            print(sub_df_sensor.head(),'\n')
            df_combined_temp_humidity[sensor_name] = sub_df_sensor
            sub_df_sensor = pd.DataFrame()
        elif sensor_name in ['Shower_Hum_Temp_humidity','Shower_Hum_Temp_temp']:
            continue
        else:
            df_sensor = data_subject_2[sensor_name][1]
            df_sensor['ts_on'] = pd.to_datetime(df_sensor['ts_on'])
            df_sensor['ts_off'] = pd.to_datetime(df_sensor['ts_off'])
            if st != None and et!=None:
                filtered_df = df_sensor[(df_sensor['ts_on'] >= start_date) & (df_sensor['ts_on'] <= end_date)]
            else:
                filtered_df = df_sensor.copy()
            sub_df_sensor = filtered_df[['sensor_id', 'ts_on', 'ts_off']]
            df_combined_non_temp_humidity.append(sub_df_sensor)
            sub_df_sensor = pd.DataFrame()

    df_combined_non_temp_humidity = pd.concat(df_combined_non_temp_humidity, ignore_index=True)
    df_combined_non_temp_humidity = df_combined_non_temp_humidity.sort_values(by=['sensor_id'], ascending=True)

    # clear_output(wait=True)

    fig = make_subplots(
        rows=3, 
        cols=1, 
        shared_xaxes=True, 
        row_heights=[0.1, 0.1, 0.4],
        vertical_spacing = 0.01
    ) 

    fig.add_trace(
        go.Scatter(
            x=df_combined_temp_humidity['Stove_Hum_Temp_temp'].reset_index().ts_datetime,
            y=df_combined_temp_humidity['Stove_Hum_Temp_temp'].reset_index().sensor_status, 
            mode='lines', 
            name='Temperature', 
            line=dict(color='red')
        ), 
        row=1, 
        col=1
    )

    fig.add_trace(
        go.Scatter(
            x=df_combined_temp_humidity['Stove_Hum_Temp_humidity'].reset_index().ts_datetime,
            y=df_combined_temp_humidity['Stove_Hum_Temp_humidity'].reset_index().sensor_status,  
            mode='lines', 
            name='Humidity', 
            line=dict(color='blue')
        ), 
        row=2, 
        col=1
    )
    fig1 = go.Figure(px.timeline(df_combined_non_temp_humidity, x_start='ts_on', x_end='ts_off', y='sensor_id', color='sensor_id',labels={'sensor_id': 'Sensor ID'}))
    for i in range(len(fig1.data)):
        fig.append_trace(fig1.data[i],row=3,col=1)


    fig.update_layout(barmode='overlay')
    fig.update_yaxes(fixedrange=True)
    fig.update_layout(height=500) 
    fig.update_xaxes(type='date')
    fig.show()        


In [54]:
with open('/home/hubble/work/serenade/src/data_matteo_upto_september_25_2024.pkl', 'rb') as file:
    data = pickle.load(file)

In [55]:
subjects = list(data.keys())
print(subjects)

['subject_1', 'subject_2', 'subject_3', 'subject_4', 'subject_5', 'subject_7', 'subject_8', 'subject_9', 'subject_10', 'subject_11', 'subject_12', 'subject_13', 'subject_14']


In [61]:

subject = subjects[1]
print(subject)
data_subject_2 = data[subject]
environmental_sensors = list(data_subject_2.keys())
st = pd.to_datetime('2024-05-01 00:00:00')
et = pd.to_datetime('2024-05-30 23:59:59')
plot_sensor_active_periods(data_subject_2, environmental_sensors, st, et)

subject_2
I came inside......
0 Dishes_Silverware
1 FoodStorage
2 Freezer
3 HouseEntrance
4 Medicines
5 MotionBathroom
6 MotionBedroom
7 MotionKitchen
8 MotionLivingroom
9 MotionOtherroom
10 PlugTvHall
11 Refrigerator
12 Shower_Hum_Temp_humidity
13 Shower_Hum_Temp_temp
14 Stove_Hum_Temp_humidity
            sensor_id             ts_datetime  sensor_status
35400  Hum_Temp_Stove 2024-05-01 00:13:29.908          44.00
35401  Hum_Temp_Stove 2024-05-01 00:13:29.946          44.00
35402  Hum_Temp_Stove 2024-05-01 00:14:50.910          44.19
35403  Hum_Temp_Stove 2024-05-01 00:14:50.940          44.19
35404  Hum_Temp_Stove 2024-05-01 01:11:31.856          43.83 

15 Stove_Hum_Temp_temp
            sensor_id             ts_datetime  sensor_status
35400  Hum_Temp_Stove 2024-05-01 00:13:29.908          23.03
35401  Hum_Temp_Stove 2024-05-01 00:13:29.946          23.03
35402  Hum_Temp_Stove 2024-05-01 00:14:50.910          23.06
35403  Hum_Temp_Stove 2024-05-01 00:14:50.940          23.06
35404  